# Fit Inspection

This notebook visually inspects light curve fits for each target in a given survey. 


In [ ]:
import sys
from pathlib import Path

import sndata
import yaml

sys.path.insert(0, '../')

from phot_class import fit_func_wraps
from phot_class import models
from phot_class import utils
from phot_class.fitting import run_band_fits, run_collective_fits


We begin by defining the data release to inspect light-curves for. We also define which parameters to vary while fitting and load a dictionary of priors from file.


In [ ]:
# Set this to the survey you want to inspect
# e.g. sndata.des.sn3yr, sndata.sdss.sako18
survey = sndata.sdss.sako18

# Specify which model parameters you want to vary while fitting
# Options: 'z', t0', 'x0', 'x1', 'c'
vparams = ['t0', 'amplitude', 'x1', 'c']

# Specify the path where to load / save prior values from
config_path = Path('../config_files/sdss_config_ext.yml')

# Specify type of fits to investigate
fitting_type = 'collective'


Next we build an iterable over the object ids for each supernova we want to fit.

In [ ]:
# Make sncosmo aware of the models and bandpasses we are using
models.register_sources(force=True)
survey.download_module_data()
survey.register_filters(force=True)

# Define an iterable over object ids
targets = survey.get_available_ids()
total_targets = len(targets)

target_iter = iter(targets)
band_names = survey.band_names
lambda_eff = survey.lambda_effective


In [ ]:
with open(config_path) as config_file:
    config = yaml.load(config_file, Loader=yaml.FullLoader)


Finally, we iterate over each supernova one at a time and fit each light-curve using the priors we loaded in from file earlier. To inspect fit results for a specific target, manually specify ``target_for_inspection`` as the desired object id.

In [ ]:
# Iterate to next target
target_for_inspection = next(target_iter)

# Or fit a given id
# target_for_inspection = '15749'

i = survey.get_available_ids().index(target_for_inspection)
print(f'{i + 1} / {total_targets}')


In [ ]:
print('Inspecting:', target_for_inspection)

# Load data. Note that we drop any data in the CSP NIR filters
# These filters tend to throw off our fits
target_data = survey.get_data_for_id(target_for_inspection)
target_data = target_data[target_data['band'] != 'csp_dr3_Ydw']
target_data = target_data[target_data['band'] != 'csp_dr3_Y']
z = target_data.meta['z']

hsiao_prior, hsiao_kwargs, sn91bg_prior, sn91bg_kwargs = \
    utils.parse_config_dict(target_for_inspection, config)

print(f'\nHsiao prior: {hsiao_prior}')
print(f'\nHsiao kwargs: {hsiao_kwargs}')
print(f'\nsn91bg prior: {sn91bg_prior}')
print(f'\nsn91bg kwargs: {sn91bg_kwargs}\n')

if fitting_type == 'band':
    fit_results = run_band_fits(
        target_data.meta['obj_id'],
        target_data,
        fit_func=fit_func_wraps.simple_fit,
        priors_hs=hsiao_prior,
        priors_bg=sn91bg_prior,
        kwargs_hs=hsiao_kwargs,
        kwargs_bg=sn91bg_kwargs,
        show_plots=True
    )

elif fitting_type == 'collective':
    fit_results = run_collective_fits(
        target_data.meta['obj_id'],
        target_data,
        fit_func=fit_func_wraps.simple_fit,
        priors_hs=hsiao_prior,
        priors_bg=sn91bg_prior,
        kwargs_hs=hsiao_kwargs,
        kwargs_bg=sn91bg_kwargs,
        show_plots=True,
        band_names=survey.band_names,
        lambda_eff=survey.lambda_effective
    )

